In [27]:
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise.dataset import BUILTIN_DATASETS
import pandas as pd

In [28]:
data = Dataset.load_from_file("../data/u.data.txt", reader=Reader(line_format="user item rating timestamp", sep="\t"),)

In [29]:
df = pd.DataFrame(data.raw_ratings, columns=['userId', 'movieId', 'rating', 'timestamp'])

In [30]:
df

,userId,movieId,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596
...,...,...,...,...
99995,880,476,3.0,880175444
99996,716,204,5.0,879795543
99997,276,1090,1.0,874795795
99998,13,225,2.0,882399156


In [31]:
print(df['movieId'].value_counts().count())
print(df['userId'].value_counts().count())

1682
943


In [32]:
df['rating'].mode()

0    4.0
Name: rating, dtype: float64

In [33]:
df_train, df_test = train_test_split(data, test_size=0.25, random_state=13)

In [34]:
len(df_test)

25000

In [62]:
from surprise import SVD, KNNBasic, accuracy

In [63]:
sim_options = {'name': 'cosine', 'user_based': False}
knn = KNNBasic(sim_options=sim_options)

In [64]:
knn.fit(df_train)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [65]:
predictions = knn.test(df_test)

In [66]:
predictions_df = pd.DataFrame(predictions)
predictions_df.head()

,uid,iid,r_ui,est,details
0,7,633,5.0,4.199452,"{'actual_k': 40, 'was_impossible': False}"
1,422,287,3.0,3.470344,"{'actual_k': 40, 'was_impossible': False}"
2,804,163,3.0,3.571674,"{'actual_k': 40, 'was_impossible': False}"
3,189,480,5.0,4.222826,"{'actual_k': 40, 'was_impossible': False}"
4,238,546,3.0,3.473417,"{'actual_k': 17, 'was_impossible': False}"


In [67]:
predictions_df[(predictions_df['uid'] == '500') & (predictions_df['iid'] == '699')]

,uid,iid,r_ui,est,details
946,500,699,3.0,3.47479,"{'actual_k': 40, 'was_impossible': False}"


In [68]:
accuracy.rmse(predictions)

RMSE: 1.0272


1.0271678039029761

In [71]:
predictions_df.sort_values(by=['est'], inplace=True, ascending = False)
predictions_df.head()

,uid,iid,r_ui,est,details
22469,849,234,5.0,4.951929,"{'actual_k': 19, 'was_impossible': False}"
1974,849,427,4.0,4.950547,"{'actual_k': 19, 'was_impossible': False}"
8272,849,568,4.0,4.949215,"{'actual_k': 19, 'was_impossible': False}"
5138,849,174,5.0,4.947691,"{'actual_k': 19, 'was_impossible': False}"
22021,688,1127,5.0,4.928412,"{'actual_k': 15, 'was_impossible': False}"


In [72]:
recom = predictions_df[predictions_df.uid =='849']['iid'].to_list()

In [74]:
sim_options = {
    'name': 'cosine',
    'user_based': True
}
knn = KNNBasic(sim_options=sim_options)
knn.fit(df_train)
predictions = knn.test(df_test)
accuracy.rmse(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0175


1.0174852296380237

In [75]:
model = SVD()
model.fit(df_train)
predictions = model.test(df_test)
accuracy.rmse(predictions)

RMSE: 0.9424


0.9424219765186304